In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import re
import os
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import joblib

import nltk

nltk.data.path.append('/Users/akashmukherjee/nltk_data')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


False

In [41]:
df=pd.read_csv('../data/final_hateXplain.csv')
df

,comment,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
2,1 0 yrs <number> white women raped by niggers ...,hatespeech,African,Nonreligious,No_gender,No_orientation,NaN
3,1 2 h ago ching chong accepted your friend req...,offensive,Asian,Nonreligious,No_gender,No_orientation,NaN
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,NaN
...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,offensive,African,Nonreligious,No_gender,No_orientation,NaN
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hatespeech,Arab,Islam,No_gender,No_orientation,NaN
20106,🤣 this so ghetto i ’ m ready to get tf,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hatespeech,Arab,Nonreligious,No_gender,No_orientation,NaN


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20109 entries, 0 to 20108
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comment             20109 non-null  object
 1   label               20109 non-null  object
 2   Race                20109 non-null  object
 3   Religion            20109 non-null  object
 4   Gender              20109 non-null  object
 5   Sexual Orientation  20109 non-null  object
 6   Miscellaneous       3533 non-null   object
dtypes: object(7)
memory usage: 1.1+ MB


In [43]:
df['Miscellaneous'].fillna('None', inplace=True)
df

/var/folders/8f/zsh7smnd15z_gk8v12njmtd00000gn/T/ipykernel_49460/1683019454.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Miscellaneous'].fillna('None', inplace=True)


,comment,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,None
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
2,1 0 yrs <number> white women raped by niggers ...,hatespeech,African,Nonreligious,No_gender,No_orientation,None
3,1 2 h ago ching chong accepted your friend req...,offensive,Asian,Nonreligious,No_gender,No_orientation,None
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,None
...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,offensive,African,Nonreligious,No_gender,No_orientation,None
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hatespeech,Arab,Islam,No_gender,No_orientation,None
20106,🤣 this so ghetto i ’ m ready to get tf,offensive,No_race,Nonreligious,No_gender,No_orientation,Other
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hatespeech,Arab,Nonreligious,No_gender,No_orientation,None


In [44]:
df['Features']=df['Race']+' '+df['Religion']+' '+df['Gender']+' '+df['Sexual Orientation']+' '+df['Miscellaneous']
df['text']=df['comment']+' '+df['Features']
df

,comment,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous,Features,text
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,None,No_race Nonreligious No_gender No_orientation ...,0 u0 lmao wow fuck you too 😂 😂 No_race Nonreli...
1,1 0 th floor maybe wow cnn with the fakenews t...,offensive,No_race,Nonreligious,No_gender,No_orientation,Other,No_race Nonreligious No_gender No_orientation ...,1 0 th floor maybe wow cnn with the fakenews t...
2,1 0 yrs <number> white women raped by niggers ...,hatespeech,African,Nonreligious,No_gender,No_orientation,None,African Nonreligious No_gender No_orientation ...,1 0 yrs <number> white women raped by niggers ...
3,1 2 h ago ching chong accepted your friend req...,offensive,Asian,Nonreligious,No_gender,No_orientation,None,Asian Nonreligious No_gender No_orientation None,1 2 h ago ching chong accepted your friend req...
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,None,No_race Nonreligious No_gender No_orientation ...,1 8 th century mayhem and lawlessness had noth...
...,...,...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,offensive,African,Nonreligious,No_gender,No_orientation,None,African Nonreligious No_gender No_orientation ...,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hatespeech,Arab,Islam,No_gender,No_orientation,None,Arab Islam No_gender No_orientation None,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...
20106,🤣 this so ghetto i ’ m ready to get tf,offensive,No_race,Nonreligious,No_gender,No_orientation,Other,No_race Nonreligious No_gender No_orientation ...,🤣 this so ghetto i ’ m ready to get tf No_race...
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hatespeech,Arab,Nonreligious,No_gender,No_orientation,None,Arab Nonreligious No_gender No_orientation None,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...


In [45]:
df['text'][1]

'1 0 th floor maybe wow cnn with the fakenews the body count is going to be waay bad too but we can all go home now they have the white ar <number> toting suicide shooter with huge drum mags dead on the 3 2 nd floor No_race Nonreligious No_gender No_orientation Other'

In [46]:
df['label'].value_counts()

label
normal        7818
hatespeech    6484
offensive     5807
Name: count, dtype: int64

In [47]:
category_mapping = {
    "normal": "normal",
    "hatespeech": "hate",
    "offensive": "hate"
    
    
}

In [48]:
df['label'] = df['label'].map(category_mapping)
df

,comment,label,Race,Religion,Gender,Sexual Orientation,Miscellaneous,Features,text
0,0 u0 lmao wow fuck you too 😂 😂,normal,No_race,Nonreligious,No_gender,No_orientation,None,No_race Nonreligious No_gender No_orientation ...,0 u0 lmao wow fuck you too 😂 😂 No_race Nonreli...
1,1 0 th floor maybe wow cnn with the fakenews t...,hate,No_race,Nonreligious,No_gender,No_orientation,Other,No_race Nonreligious No_gender No_orientation ...,1 0 th floor maybe wow cnn with the fakenews t...
2,1 0 yrs <number> white women raped by niggers ...,hate,African,Nonreligious,No_gender,No_orientation,None,African Nonreligious No_gender No_orientation ...,1 0 yrs <number> white women raped by niggers ...
3,1 2 h ago ching chong accepted your friend req...,hate,Asian,Nonreligious,No_gender,No_orientation,None,Asian Nonreligious No_gender No_orientation None,1 2 h ago ching chong accepted your friend req...
4,1 8 th century mayhem and lawlessness had noth...,normal,No_race,Nonreligious,No_gender,No_orientation,None,No_race Nonreligious No_gender No_orientation ...,1 8 th century mayhem and lawlessness had noth...
...,...,...,...,...,...,...,...,...,...
20104,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...,hate,African,Nonreligious,No_gender,No_orientation,None,African Nonreligious No_gender No_orientation ...,😂 😂 😂 😂 😂 y ’ all be so mad at caucasians fail...
20105,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...,hate,Arab,Islam,No_gender,No_orientation,None,Arab Islam No_gender No_orientation None,🤔 🤔 🤔 🤔 mmm yes the trebuchet memes are much n...
20106,🤣 this so ghetto i ’ m ready to get tf,hate,No_race,Nonreligious,No_gender,No_orientation,Other,No_race Nonreligious No_gender No_orientation ...,🤣 this so ghetto i ’ m ready to get tf No_race...
20107,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...,hate,Arab,Nonreligious,No_gender,No_orientation,None,Arab Nonreligious No_gender No_orientation None,🤣 🤣 😂 😂 🤣 🤣 😂 😂 🤣 🤣 you spelled my name wrong ...


In [49]:
df.isna().sum()

comment               0
label                 0
Race                  0
Religion              0
Gender                0
Sexual Orientation    0
Miscellaneous         0
Features              0
text                  0
dtype: int64

In [50]:
def preprocess(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    
    # Remove hashtags and mentions
    tokens = [re.sub(r'#\S+', '', token) for token in tokens]
    tokens = [re.sub(r'@\S+', '', token) for token in tokens]
    
    # Remove emojis
    tokens = [token for token in tokens if not emoji.is_emoji(token)]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Join tokens back into a single string
    text = ' '.join(tokens)
    
    return text

In [51]:
df['text']=df['text'].apply(preprocess)

In [52]:
X = df['text']
y = df['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
X_train

16006    thank think white spanairds white hence hispan...
5394     stupid nigger ass still posting unsecured link...
17228    dude sits around day trying argue anglin other...
9102     exhausted queers coloreds saying shit like yes...
11651    jasmine tweeted man job still got tweet told d...
                               ...                        
11284    islam antisexist parading together like nazi j...
11964    keverc  number  hours ago hey george hope enjo...
5390     niggers also said martian manhunter beat visio...
860       user   user  hi hate asians asian nonreligiou...
15795    stupid commie cunts fuck well actually fuck ba...
Name: text, Length: 16087, dtype: object

In [54]:
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train)
X_val_tfidf=tfidf.transform(X_val)

In [55]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [56]:
model=LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf,y_train_encoded)

LogisticRegression(max_iter=1000)

In [57]:
predictions=model.predict(X_val_tfidf)
accuracy_score(y_true=y_val_encoded,y_pred=predictions)

0.8316757831924416

In [58]:
model=RandomForestClassifier(n_estimators=400)
model.fit(X_train_tfidf,y_train_encoded)

RandomForestClassifier(n_estimators=400)

In [59]:
predictions=model.predict(X_val_tfidf)
accuracy_score(y_true=y_val_encoded,y_pred=predictions)

0.8637493784186971

In [60]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

In [61]:
# Set up GridSearchCV
rs_cv = RandomizedSearchCV(estimator=model, param_distributions=param_grid, 
                           cv=5, n_jobs=-1,  scoring='accuracy')

In [62]:
rs_cv.fit(X_train_tfidf,y_train_encoded)

/Users/akashmukherjee/Programming/Practise ML/Cyberbully /my_env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/akashmukherjee/Programming/Practise ML/Cyberbully /my_env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/akashmukherjee/Programming/Practise ML/Cyberbully /my_env/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/Users/akashmukherjee/Pro

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=400),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy')

In [63]:
best_params=rs_cv.best_params_
rs_cv.best_score_

np.float64(0.7752845597879352)

In [64]:
best_params

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [65]:
os.makedirs('../models',exist_ok=True)

In [66]:
joblib.dump(model,'../models/model.joblib')

['../models/model.joblib']

In [67]:
joblib.dump(label_encoder,'../models/encoder.joblib')

['../models/encoder.joblib']